<a href="https://colab.research.google.com/github/Ykiiii/t-less_toolkit/blob/master/yk_pytless_20231130.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# colab

In [1]:
# 挂载GD
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 挂载github
using_colab = True
if using_colab:
    import torch
    import torchvision
    print("PyTorch version:", torch.__version__)
    print("Torchvision version:", torchvision.__version__)
    print("CUDA is available:", torch.cuda.is_available())
    import sys
    !{sys.executable} -m pip install vispy
    !{sys.executable} -m pip install 'git+https://github.com/Ykiiii/t-less_toolkit.git'


In [3]:
!git clone https://github.com/Ykiiii/t-less_toolkit.git


Cloning into 't-less_toolkit'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 224 (delta 5), reused 16 (delta 3), pack-reused 202
Receiving objects: 100% (224/224), 1.79 MiB | 9.52 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
/content/t-less_toolkit/t-less_download.py

In [11]:
!python t-less_toolkit/t-less_download.py --help

usage: t-less_download.py [-h] [--destination DESTINATION] [--train [obj_id ...]]
                          [--test [scene_id ...]]
                          [--sensors {primesense,kinect,canon} [{primesense,kinect,canon} ...]]
                          [--models [{cad,cad_subdivided,reconst} ...]]

Downloads and unpacks the selected parts of the T-LESS dataset. Example usage: "t-less_download.py
--train --test --models --sensors primesense" - downloads and unpacks all models and all images
from the Primesense sensor.

options:
  -h, --help            show this help message and exit
  --destination DESTINATION
                        destination folder inside which a new folder for the dataset is created
                        (default is the current folder)
  --train [obj_id ...]  IDs of objects (all are downloaded if no ID is specified)
  --test [scene_id ...]
                        IDs of scenes (all are downloaded if no ID is specified)
  --sensors {primesense,kinect,canon} [{pri

In [ ]:
!echo -e '\n' | python t-less_toolkit/t-less_download.py --train  --test  --models cad --sensors primesense



In [21]:
!python /content/t-less_toolkit/check_poses_train_imgs.py

Traceback (most recent call last):
  File "/content/t-less_toolkit/check_poses_train_imgs.py", line 9, in <module>
    from pytless import inout, renderer, misc
  File "/content/t-less_toolkit/pytless/renderer.py", line 9, in <module>
    app.use_app('PyGlet') # Set backend
  File "/usr/local/lib/python3.10/dist-packages/vispy/app/_default_app.py", line 47, in use_app
    default_app = Application(backend_name)
  File "/usr/local/lib/python3.10/dist-packages/vispy/app/application.py", line 47, in __init__
    self._use(backend_name)
  File "/usr/local/lib/python3.10/dist-packages/vispy/app/application.py", line 240, in _use
    raise RuntimeError(msg)
RuntimeError: Could not import backend "Pyglet":
No module named 'pyglet'


In [ ]:
!zip -r /content/drive/MyDrive/Dataset/t-less_v2.zip /content/t-less_v2

# test

In [ ]:
from pytless import inout, renderer, misc
import os
import numpy as np
import scipy.misc
import matplotlib.pyplot as plt
import imageio

In [ ]:
scene_ids = [20] # Choose which scene_ids to render. Eg. range(1, 21)
device = 'primesense' # options: 'primesense', 'kinect', 'canon'
model_type = 'cad' # options: 'cad', 'reconst'
im_step = 100 # Consider every im_step-th image

# Path to the T-LESS dataset.
# Which you can download using the t-less_download.py script.
data_path = 'E:\\DataSets'

# Path to the folder in which the images produced by this script will be saved
output_dir = os.path.join(data_path, 'output_check_poses_test_imgs')

# Paths to the elements of the T-LESS dataset
model_path_mask = os.path.join(data_path, 'models_' + model_type, 'obj_{:02d}.ply')
scene_info_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'info.yml')
scene_gt_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'gt.yml')
rgb_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'rgb', '{:04d}.{}')
depth_path_mask = os.path.join(data_path, 'test_{}', '{:02d}', 'depth', '{:04d}.png')
rgb_ext = {'primesense': 'png', 'kinect': 'png', 'canon': 'jpg'}
obj_colors_path = os.path.join('data', 'obj_rgb.txt')
vis_rgb_path_mask = os.path.join(output_dir, '{:02d}_{}_{}_{:04d}_rgb.png')
vis_depth_path_mask = os.path.join(output_dir, '{:02d}_{}_{}_{:04d}_depth_diff.png')

misc.ensure_dir(output_dir)
obj_colors = inout.load_colors(obj_colors_path)